In [2]:
# ===================================
# Retail Banking Analysis Project
# Author: Omolara Okiki
# Date: 02-10-2025
# ===================================

### Workflow:
##### 1. Data Ingestion
##### 2. Data Cleaning
##### 3. Exploratory Data Analysis
##### 4. RFM Segmentation
##### 5. Save Outputs



In [5]:
# Import essential Libraries

import pandas as pd                 # for data manipulations
import numpy as np                  # for numerical operations
import matplotlib. pyplot as plt    # for data visualisation
import seaborn as sns               # for statistical graphics      
import datetime as dt               # date handling
import warnings                     # suppress warnings

#ignore unnecessary warning messages
warnings.filterwarnings("ignore")

# Data Loading and Initial Inspection

In [6]:
data = pd.read_csv("../data/raw/bank_data_C.csv")

In [7]:
# View data
data.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1041614 entries, 0 to 1041613
Data columns (total 9 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   TransactionID            1041614 non-null  object 
 1   CustomerID               1041614 non-null  object 
 2   CustomerDOB              1041614 non-null  object 
 3   CustGender               1041614 non-null  object 
 4   CustLocation             1041614 non-null  object 
 5   CustAccountBalance       1041614 non-null  float64
 6   TransactionDate          1041614 non-null  object 
 7   TransactionTime          1041614 non-null  int64  
 8   TransactionAmount (INR)  1041614 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 71.5+ MB


In [9]:
data_raw = data.copy()

In [10]:
data.describe(include= "all")

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
count,1041614,1041614,1041614,1041614,1041614,1.041614e+06,1041614,1.041614e+06,1.041614e+06
unique,1041614,879358,17233,3,9275,NaN,55,NaN,NaN
top,T1048567,C5533885,1/1/1800,M,MUMBAI,NaN,7/8/16,NaN,NaN
freq,1,6,56292,760978,101997,NaN,27084,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,1.149986e+05,NaN,1.571221e+05,1.566096e+03
std,NaN,NaN,NaN,NaN,NaN,8.467609e+05,NaN,5.126352e+04,6.561464e+03
min,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,0.000000e+00,0.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,4.728140e+03,NaN,1.240410e+05,1.600000e+02
50%,NaN,NaN,NaN,NaN,NaN,1.676852e+04,NaN,1.642470e+05,4.575000e+02
75%,NaN,NaN,NaN,NaN,NaN,5.742885e+04,NaN,2.000220e+05,1.200000e+03


In [11]:
# Column Summary

for col in data.columns:
    print(f"--- {col} ---")
    print("Data type:", data[col].dtype)
    print("Missing values:", data[col].isnull().sum())
    print("Unique values:", data[col].nunique())
    print("Top 5 values:\n", data[col].value_counts().head(5))
    print("\n")


--- TransactionID ---
Data type: object
Missing values: 0
Unique values: 1041614
Top 5 values:
 TransactionID
T1048567    1
T1          1
T2          1
T3          1
T4          1
Name: count, dtype: int64


--- CustomerID ---
Data type: object
Missing values: 0
Unique values: 879358
Top 5 values:
 CustomerID
C5533885    6
C6624352    6
C4327447    6
C6222360    6
C4021562    6
Name: count, dtype: int64


--- CustomerDOB ---
Data type: object
Missing values: 0
Unique values: 17233
Top 5 values:
 CustomerDOB
1/1/1800    56292
1/1/89        809
1/1/90        784
6/8/91        698
1/1/91        665
Name: count, dtype: int64


--- CustGender ---
Data type: object
Missing values: 0
Unique values: 3
Top 5 values:
 CustGender
M    760978
F    280635
T         1
Name: count, dtype: int64


--- CustLocation ---
Data type: object
Missing values: 0
Unique values: 9275
Top 5 values:
 CustLocation
MUMBAI       101997
NEW DELHI     84143
BANGALORE     81330
GURGAON       73594
DELHI         70549
Na

## DATA CLEANING  
##### Check for missing Data  
##### Convert Dates Fields: CustomerDOB and TransactionDate  
##### Derive Age: CustomerDOB  
##### Standardize Features: Location and Gender, and TransactionTime  
##### Check for Duplicates  
##### Check for Outliers: High values in Transaction amount and CustomerAccountBalance 

In [12]:
# Check for missing values
data.isnull().sum()
 

TransactionID              0
CustomerID                 0
CustomerDOB                0
CustGender                 0
CustLocation               0
CustAccountBalance         0
TransactionDate            0
TransactionTime            0
TransactionAmount (INR)    0
dtype: int64

### Summary: No missing data, all data types are accurate except Transaction date and CustomerDOB, which needs to be converted 

In [13]:
# Clean blanks/NA-likes
for c in ['TransactionDate','CustomerDOB']:
    data[c] = (data[c].astype(str).str.strip()
               .replace({'': pd.NA, 'NA': pd.NA, 'N/A': pd.NA, 'null': pd.NA}, regex=False))

In [14]:
# Convert relevant column to datetime
data['TransactionDate'] = pd.to_datetime(data['TransactionDate'], errors='coerce')
data['CustomerDOB']     = pd.to_datetime(data['CustomerDOB'], errors='coerce')
 

In [15]:
# Subtract 100 years for DOBs in the future - to fix years after the current 
today = pd.Timestamp.today()
future_mask = data['CustomerDOB'] > today
data.loc[future_mask, 'CustomerDOB'] = data.loc[future_mask, 'CustomerDOB'] - pd.DateOffset(years=100)


In [16]:
#  fix years before 1900
too_old_mask = data['CustomerDOB'].dt.year < 1900
data.loc[too_old_mask, 'CustomerDOB'] = data.loc[too_old_mask, 'CustomerDOB'] + pd.DateOffset(years=100)


In [17]:
# Create "Age" from CustomerDOB
today = pd.to_datetime("today")
data['Age'] = (today - data['CustomerDOB']).dt.days// 365

In [18]:
data.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),Age
0,T1,C5841053,1994-10-01,F,JAMSHEDPUR,17819.05,2016-02-08,143207,25.0,31
1,T2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-02-08,141858,27999.0,68
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-02-08,142712,459.0,28
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-02-08,142714,2060.0,52
4,T5,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-02-08,181156,1762.5,37


In [19]:
# Investigate the Ages column for abnormal figures
data['Age'].describe()


count    1.041614e+06
mean     4.445848e+01
std      2.105150e+01
min      0.000000e+00
25%      3.400000e+01
50%      3.800000e+01
75%      4.500000e+01
max      1.250000e+02
Name: Age, dtype: float64

In [20]:
data['CustomerDOB'].unique()


<DatetimeArray>
['1994-10-01 00:00:00', '1957-04-04 00:00:00', '1996-11-26 00:00:00',
 '1973-09-14 00:00:00', '1988-03-24 00:00:00', '1972-08-10 00:00:00',
 '1992-01-26 00:00:00', '1982-01-27 00:00:00', '1988-04-19 00:00:00',
 '1984-06-22 00:00:00',
 ...
 '1957-10-13 00:00:00', '1944-07-30 00:00:00', '2005-08-15 00:00:00',
 '1946-06-04 00:00:00', '1950-07-01 00:00:00', '2015-11-18 00:00:00',
 '1945-06-28 00:00:00', '1965-07-18 00:00:00', '1942-05-15 00:00:00',
 '1944-10-24 00:00:00']
Length: 17233, dtype: datetime64[ns]

In [21]:
# Check rows with negative age
data[data['Age'] < 18]

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),Age
387,T391,C4417679,2021-01-19,M,NEW DELHI,1125922.25,2016-10-21,202654,4570.00,4
1324,T1329,C6417656,2021-01-19,M,NEW DELHI,1125922.25,2016-10-21,192755,6300.00,4
1662,T1668,C4217631,2021-01-19,M,NEW DELHI,1125922.25,2016-10-21,195748,2460.00,4
8129,T8174,C1017880,2013-02-20,M,DELHI,20257.22,2016-09-25,191750,359.52,12
12467,T12532,C5333741,2010-10-18,M,FARIDABAD,29225.01,2016-09-27,160248,500.00,15
...,...,...,...,...,...,...,...,...,...,...
1001765,T1008538,C3017630,2021-01-19,M,NEW DELHI,1125922.25,2016-09-13,212632,4285.00,4
1012982,T1019804,C4420589,2014-01-13,M,GHAZIABAD,751220.52,2016-09-13,203924,1849.00,11
1015616,T1022448,C4825367,2008-12-05,F,NEW DELHI,79984.60,2016-09-13,185905,285.00,16
1028779,T1035679,C8528348,2012-07-03,M,NASHIK,26000.92,2016-09-13,102620,570.00,13


In [22]:
data.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),Age
0,T1,C5841053,1994-10-01,F,JAMSHEDPUR,17819.05,2016-02-08,143207,25.0,31
1,T2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-02-08,141858,27999.0,68
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-02-08,142712,459.0,28
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-02-08,142714,2060.0,52
4,T5,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-02-08,181156,1762.5,37


In [23]:
# Keep Only rows where Age is between 1 and 100
data1 = data[(data['Age'] > 18) & (data['Age'] <= 100)]


In [24]:
# Rename columns if they still have (INR)
data1.rename(columns={
    'TransactionDate (INR)': 'TransactionDate',
    'TransactionAmount (INR)': 'TransactionAmount'
}, inplace=True)


In [28]:
data1.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount,Age,time_length
0,T1,C5841053,1994-10-01,F,JAMSHEDPUR,17819.05,2016-02-08,143207,25.0,31,6
1,T2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-02-08,141858,27999.0,68,6
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-02-08,142712,459.0,28,6
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-02-08,142714,2060.0,52,6
4,T5,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-02-08,181156,1762.5,37,6


In [27]:
#Check TransactionTime Format
data1['TransactionTime'].dropna().astype(str).unique()[:50]


array(['143207', '141858', '142712', '142714', '181156', '173940',
       '173806', '170537', '192825', '192446', '204133', '205108',
       '203834', '84706', '82253', '125725', '124734', '122135', '152821',
       '152824', '105329', '125629', '124834', '160533', '160537',
       '160544', '160642', '160558', '160750', '182714', '182715',
       '182915', '182918', '180948', '183614', '132517', '221111',
       '220926', '222006', '222008', '222609', '223146', '223149',
       '100019', '133136', '133139', '133141', '141853', '141704',
       '165658'], dtype=object)

In [29]:
# Keep only digits, ensure length 6, take the rightmost 6 in case of longer strings
tstr = (data1['TransactionTime'].astype(str)
          .str.replace(r'\D', '', regex=True)  # remove non-digits
          .str.zfill(6)                        # pad left
          .str[-6:])                           # keep last 6 chars

data1['TransactionTime'] = pd.to_datetime(
    tstr, format='%H%M%S', errors='coerce'
).dt.time



In [30]:
data1.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount,Age,time_length
0,T1,C5841053,1994-10-01,F,JAMSHEDPUR,17819.05,2016-02-08,14:32:07,25.0,31,6
1,T2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-02-08,14:18:58,27999.0,68,6
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-02-08,14:27:12,459.0,28,6
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-02-08,14:27:14,2060.0,52,6
4,T5,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-02-08,18:11:56,1762.5,37,6


In [31]:
# Combine date and time into a new datetime column
data1['TransactionDateandTime'] = pd.to_datetime(
    data1['TransactionDate'].dt.strftime('%Y-%m-%d') + ' ' +
    pd.to_datetime(data1['TransactionTime'].astype(str), errors='coerce').dt.strftime('%H:%M:%S'),
    errors='coerce'
)


In [32]:
data1.head(20)


,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount,Age,time_length,TransactionDateandTime
0,T1,C5841053,1994-10-01,F,JAMSHEDPUR,17819.05,2016-02-08,14:32:07,25.00,31,6,2016-02-08 14:32:07
1,T2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-02-08,14:18:58,27999.00,68,6,2016-02-08 14:18:58
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-02-08,14:27:12,459.00,28,6,2016-02-08 14:27:12
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-02-08,14:27:14,2060.00,52,6,2016-02-08 14:27:14
4,T5,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-02-08,18:11:56,1762.50,37,6,2016-02-08 18:11:56
5,T6,C1536588,1972-08-10,F,ITANAGAR,53609.20,2016-02-08,17:39:40,676.00,53,6,2016-02-08 17:39:40
6,T7,C7126560,1992-01-26,F,MUMBAI,973.46,2016-02-08,17:38:06,566.00,33,6,2016-02-08 17:38:06
7,T8,C1220223,1982-01-27,M,MUMBAI,95075.54,2016-02-08,17:05:37,148.00,43,6,2016-02-08 17:05:37
8,T9,C8536061,1988-04-19,F,GURGAON,14906.96,2016-02-08,19:28:25,833.00,37,6,2016-02-08 19:28:25
9,T10,C6638934,1984-06-22,M,MUMBAI,4279.22,2016-02-08,19:24:46,289.11,41,6,2016-02-08 19:24:46


In [36]:
# Save raw/cleaned dataset into a 'data' folder  
data.to_csv('../data/raw/raw_transactions.csv', index=False)
data1.to_csv('../data/processed/cleaned_transactions.csv', index=False)

In [38]:
import os
print(os.listdir('../data/raw'))

['bank_data_C.csv', 'raw_transactions.csv']
